In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os 
import sys 
import torch 
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import lightning.pytorch as pl


sys.path.append('/Users/ivan_zorin/Documents/AIRI/code/ntl/')

from models import *
from data.data import SGCCDataset, sgcc_train_test_split
from utils.utils import load_config

In [3]:
config = load_config('../configs/local_config.yaml')
config

namespace(ntl_wd='/Users/ivan_zorin/Documents/AIRI/code/ntl/',
          data_path='/Users/ivan_zorin/Documents/AIRI/data/sgcc/data.csv',
          log_path='',
          n_epochs=100,
          batch_size=16,
          lr=0.0001,
          device='cpu')

In [15]:
train_data, val_data, test_data = sgcc_train_test_split(config)

train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, num_workers=1)
val_loader = DataLoader(val_data, batch_size=config.batch_size, shuffle=False, num_workers=1)
test_loader = DataLoader(test_data, batch_size=config.batch_size, shuffle=False,  num_workers=1)


In [16]:
lstmae = LSTMAE_old(input_size=1, hidden_size=[1024], n_lstms=1)

model = LitLSTMAE(
    model=lstmae,
    loss_fn=F.l1_loss,
    optimizer=torch.optim.Adam,
    config=config
    
)

In [17]:
trainer = pl.Trainer(fast_dev_run=True,
                     max_steps=5,
                     num_sanity_val_steps=1,
                     accelerator=config.device,
)


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ivan_zorin/opt/miniconda3/envs/pytorch/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [18]:
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
    
)


  | Name  | Type       | Params
-------------------------------------
0 | model | LSTMAE_old | 4.2 M 
-------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.843    Total estimated model params size (MB)
/Users/ivan_zorin/opt/miniconda3/envs/pytorch/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/ivan_zorin/opt/miniconda3/envs/pytorch/lib/python3.9/site-packages/lightning/pytorch/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs fo

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


[]